## Random Forest

Para o algoritimo de random forest, o bagging (bootstrap aggregating) é 90% do processo. o que falta para completar o random forest é a aleatoriedade na seleção das features (variáveis) para cada árvore.
1. Bagging, passo a passo:
   - **Criação de subconjuntos de dados**: A partir do conjunto de dados original, são criados vários subconjuntos de dados.
   - **Seleção aleatória de features**: Para cada árvore, um subconjunto aleatório de features é selecionado. Isso significa que, em vez de considerar todas as variáveis para dividir os nós da árvore, apenas um subconjunto delas é considerado.
   - **Treinamento de modelos**: Cada subconjunto é usado para treinar um modelo independente.
   - **Combinação dos resultados**: Após o treinamento, os resultados dos modelos são combinados. Para problemas de classificação, a combinação é feita por votação majoritária. Para problemas de regressão, usamos a média dos resultados.

2. Random Forest é uma extensão do bagging que introduz ainda mais aleatoriedade no processo de treinamento. Em vez de treinar cada árvore em um subconjunto aleatório de dados, o Random Forest também seleciona aleatoriamente um subconjunto de features para cada divisão em cada árvore. Isso ajuda a tornar as árvores mais independentes e, em última análise, melhora a generalização do modelo.

### Vamos seguir com a implementação do Bagging manualmente, sem usar bibliotecas específicas para isso.


In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import plotly.express as px

#### Aqui vamos utilizar o conjunto de dados `digits` do `sklearn`, que é um conjunto de dados clássico para classificação de dígitos manuscritos.


In [2]:
digits = load_digits()

X = digits.data
y = digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

modelo = DecisionTreeClassifier(random_state=4)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)
print(f'A precisão do modelo no conjunto de treino: {modelo.score(X_train, y_train)*100:.2f} %')
print(f'A precisão do modelo no conjunto de teste: {modelo.score(X_test, y_test)*100:.2f} %')

A precisão do modelo no conjunto de treino: 100.00 %
A precisão do modelo no conjunto de teste: 86.11 %


Como visto acima, fizemos uma arvore simples de decisão para comparação.

Como podemos ver abaixo, no desultado abaixo, temos uma coluna com o voto majoritário de cada linha, que é a previsão final do modelo de Bagging.

Mas vamos implementar e definir algumas funções para realizar o Bagging de maneira mais prática. 

#### Com todos os dados necessários vamos definir as funções e implementar esse bagging usando 1000 modelos de árvore de decisão (Decision Tree Classifier) como modelos base.

Aqui temos as funções para facilitar nosso processo de Bagging:

In [3]:
def gerar_indices_bootstrap(base_treino):
    return np.random.randint(
        low=0,
        high=len(base_treino),
        size=len(base_treino)
    )

# Essa função treina um modelo de árvore de decisão usando os índices bootstrap fornecidos
def gerar_modelo_arvore_decisao(X_treino, y_treino, indices_bootstrap):
    df_bootstrap = pd.DataFrame(X_treino[indices_bootstrap])
    modelo = DecisionTreeClassifier(random_state=4)
    modelo.fit(df_bootstrap, y_treino[indices_bootstrap])
    return modelo

# Aqui está a função modificada para o Random Forest
def  gerar_modelo_random_forest(X_treino, y_treino, indices_bootstrap):
    df_bootstrap = pd.DataFrame(X_treino[indices_bootstrap])
    modelo = DecisionTreeClassifier(random_state=4, max_features='sqrt')
    modelo.fit(df_bootstrap, y_treino[indices_bootstrap])
    return modelo

def gerar_dataframe_previsoes(modelo, X_teste, nome_coluna):
    y_pred = modelo.predict(X_teste)
    return pd.Series(y_pred, name=nome_coluna)


Aqui nos fazemos a mágica acontecer:
- A função `bootstrap_sample` cria um subconjunto de dados com reposição.
- A função `train_base_model` treina um modelo de árvore de decisão em um subconjunto de dados.
- A função `bagging_predict` faz previsões combinadas usando votação majoritária a partir dos modelos treinados.

### A diferença para o random forest vem na função `DecisionTreeClassifier`, onde podemos definir o parâmetro `max_features` para selecionar aleatoriamente um subconjunto de features para cada árvore.

Vamos implementar o bagging e também o random forest manualmente para compararmos os resultados.

In [4]:
lista_previsoes_bagging = []

for i in range(1000):
    indices_bootstrap = gerar_indices_bootstrap(X_train)
    modelo_bag = gerar_modelo_arvore_decisao(X_train, y_train, indices_bootstrap)
    nome_coluna = f'modelo_{i:03}'
    previsoes_series = gerar_dataframe_previsoes(modelo_bag, X_test, nome_coluna)

    lista_previsoes_bagging.append(previsoes_series)

df_bagging = pd.concat(lista_previsoes_bagging, axis=1)
df_bagging['voto_maj'] = df_bagging.mode(axis=1)[0]
df_bagging

,modelo_000,modelo_001,modelo_002,modelo_003,modelo_004,modelo_005,modelo_006,modelo_007,modelo_008,modelo_009,...,modelo_991,modelo_992,modelo_993,modelo_994,modelo_995,modelo_996,modelo_997,modelo_998,modelo_999,voto_maj
0,8,4,8,7,8,4,8,4,5,4,...,8,8,8,8,4,4,4,1,4,4
1,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,4,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,5,5,5,5,5,5,5,5,5,...,5,5,2,5,5,5,5,5,5,5
4,2,3,1,3,6,1,3,3,9,1,...,3,3,3,3,3,2,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,8,8,8,8,8,8,8,8,8,...,8,2,8,8,8,8,2,8,2,8
356,5,5,5,5,5,5,5,5,5,5,...,5,7,5,5,5,5,5,5,5,5
357,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,7,4
358,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [5]:
lista_previsoes_random_forest = []

for i in range(1000):
    indices_bootstrap = gerar_indices_bootstrap(X_train)
    modelo_rf = gerar_modelo_random_forest(X_train, y_train, indices_bootstrap)
    nome_coluna = f'modelo_{i:03}'
    previsoes_series = gerar_dataframe_previsoes(modelo_rf, X_test, nome_coluna)

    lista_previsoes_random_forest.append(previsoes_series)

df_rf = pd.concat(lista_previsoes_random_forest, axis=1)
df_rf['voto_maj'] = df_rf.mode(axis=1)[0]
df_rf

,modelo_000,modelo_001,modelo_002,modelo_003,modelo_004,modelo_005,modelo_006,modelo_007,modelo_008,modelo_009,...,modelo_991,modelo_992,modelo_993,modelo_994,modelo_995,modelo_996,modelo_997,modelo_998,modelo_999,voto_maj
0,9,1,1,6,1,6,1,8,7,6,...,1,4,5,4,7,1,6,5,4,8
1,7,4,7,7,7,9,1,8,9,7,...,7,7,7,7,8,1,7,7,7,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,5,5,9,5,5,5,8,5,7,...,5,5,5,9,5,5,6,5,5,5
4,2,3,0,3,3,3,5,3,1,3,...,2,3,1,5,9,0,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,8,8,8,2,3,8,8,8,8,...,8,1,8,8,8,8,8,8,8,8
356,5,5,0,6,5,5,6,6,5,5,...,5,5,5,5,5,5,5,5,5,5
357,4,4,4,4,4,4,4,4,4,4,...,4,4,7,4,4,4,4,4,4,4
358,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


Por fim temos um modelo de Bagging completo, que podemos usar para treinar e fazer previsões em novos dados.
Vamos relembrar quais foram os resultados do modelo de árvore de decisão simples:

In [6]:
print(f'A precisão do modelo de arvore simples no conjunto de treino: {modelo.score(X_train, y_train)*100:.2f} %')
print(f'A precisão do modelo de arvore simples no conjunto de teste: {modelo.score(X_test, y_test)*100:.2f} %')

A precisão do modelo de arvore simples no conjunto de treino: 100.00 %
A precisão do modelo de arvore simples no conjunto de teste: 86.11 %


No df_bagging temos as previsões de cada modelo e a coluna final com a previsão do Bagging.
Vamos fazer a avaliação do modelo de Bagging:

In [7]:
df_previsao_bagging = pd.DataFrame()
df_previsao_bagging['bagging'] = df_bagging['voto_maj'].copy()
df_previsao_bagging['y_teste'] = y_test
df_previsao_bagging['acerto'] = np.where(df_previsao_bagging['bagging'] == df_previsao_bagging['y_teste'], 1, 0)
df_previsao_bagging['acerto'].mean()
print(f'A precisão dos modelos usando bagging: {df_previsao_bagging["acerto"].mean()*100:.2f} %')

df_previsao_rf = pd.DataFrame()
df_previsao_rf['random_forest'] = df_rf['voto_maj'].copy()
df_previsao_rf['y_teste'] = y_test
df_previsao_rf['acerto'] = np.where(df_previsao_rf['random_forest'] == df_previsao_rf['y_teste'], 1, 0)
df_previsao_rf['acerto'].mean()
print(f'A precisão dos modelos usando random forest: {df_previsao_rf["acerto"].mean()*100:.2f} %')


A precisão dos modelos usando bagging: 96.39 %
A precisão dos modelos usando random forest: 98.61 %


### O resultado final do Bagging é uma melhoria significativa na precisão do modelo em comparação com a árvore de decisão simples, demonstrando a eficácia do Bagging na redução do overfitting e na melhoria da generalização do modelo.

In [8]:
a = modelo.score(X_test, y_test)*100
b = df_previsao_bagging["acerto"].mean()*100
c = df_previsao_rf["acerto"].mean()*100
d = np.abs(b - a)
print(f'Acurácia Árvore Simples: {a:.2f}%\nAcurácia Bagging: {b:.2f}%\nGanho de acurácia: {d:.2f}%\n\nAcurácia Random Forest: {c:.2f}%\nGanho de acurácia: {np.abs(c - a):.2f}%')

fig = px.bar(
    x=['Árvore de Decisão Simples', 'Bagging', 'Random Forest'],
    y=[a, b, c],
    title=f'Comparação de Modelos<br>Ganho de acurácia: {d:.2f}%',
    labels={'x': 'Modelo', 'y': 'Acurácia (%)'},
    template='plotly_dark'
)
fig.add_hline(
    y=a,
    line_dash='dot',
    line_color='red',
    annotation_text=f'Linha<br>Árvore Simples: <br>{a:.2f}%', # <br> para quebra de linha
    annotation_position='top left',
    annotation_font_color='white'
)
fig.update_traces(marker_color=['blue', 'orange'])
fig.update_layout(width=500, height=600)
fig.show()

Acurácia Árvore Simples: 86.11%
Acurácia Bagging: 96.39%
Ganho de acurácia: 10.28%

Acurácia Random Forest: 98.61%
Ganho de acurácia: 12.50%
